# Modify from HMM.ipynb

### Idea: for each state(chords) , we will classify the output notes as "inside the chord" and "outside the chord" by calling note_2_class()
### as a result, each state only got 2 possible emission , and the emission matrix is [α,β] , where  α as the probability to emit note inside a chord, and β is (1-α).

### The likelihood of observing chord X and notes_sequence M , with #A notes inside the chord and #B notes outside the chord will be P(M|X) = α(x)^a * β(x)^b. where α(x)is the probability α of chord X

<br>

#### 1 New Function: note_2_class()
<pre>
paras:
    chord: str
        the current chord name 
    notes_at_t: 1D list
        observation of notes at time t
    chord_notes: Dictionary from keychorddict.json
        chord and key mapping
</pre>
#### 3 Modifications:     __(All modifications below are marked by #Modification above the line)__
1. how to initalize emission matrix
<br>Assume all chords will share same probability to emit notes inside itself. This assumption might be broken after training, such that, each chord will hv different probability to emit note inside itself
2. how to calculate likelihood
<br>given any observation at a timestamp , to calculate the likelihood of the observation[t], one's need first convert notes to 2 class , so that it can map with the emission matrix
3. how to update emission matrix
<br>When updating emission matrix, the #occurence of notes inside/outside the chord is also taken into account

In [1]:
import numpy as np
import random
import itertools
import math

In [2]:
# function to distingish whether notes in given timestamp and chord are within or outside the chord (0=outside,1=within)
import json
with open('../modules/json_files/keychorddict.json') as json_file:
    chord_notes = json.load(json_file)
  
    def note_2_class(chord,notes_at_t,chord_notes=chord_notes):
        note_in_chord=chord_notes[chord]['idx']
        return [int(note in note_in_chord) for note in notes_at_t]

In [3]:
### I threat the total length of time as the length of the observered list

class HMM:
    def __init__(self,no_of_state,no_of_value,states,values):
        #randomize all matrix, each row should sum to 1
        #self.emssion_matrix=np.array([ran/ran.sum() for ran in np.array([np.random.rand(no_of_value) for i in range(no_of_state)])]) #b[i][O]  --> probability to emit values[O] from states[i]
        

        #Modification: 2 parameter for all chord  [note inside chord & note outside chord]
        self.emssion_matrix=np.random.rand(no_of_value)
        self.emssion_matrix=self.emssion_matrix/self.emssion_matrix.sum()
        self.emssion_matrix=np.array([self.emssion_matrix,]*len(states)) # assume same probability to emit "note within chord"  for all chords
        
        self.initial_matrix= np.random.rand(no_of_state) #π[i] --> probability to start at states[i]
        self.initial_matrix/= self.initial_matrix.sum()
        
        self.transition_matrix=np.array([ran/ran.sum() for ran in np.array([np.random.rand(no_of_state) for i in range(no_of_state)])])  #a[i][j] --> probability from states[i] transit to states[j]
        
        self.states=states
        self.values=values
        self.observered=None
        
        #self.prev_initial_matrix=self.initial_matrix
        #self.prev_emssion_matrix=self.emssion_matrix
        #self.prev_transition_matrix=self.transition_matrix
        
    def debug(self):
        print('initial_matrix\n',self.initial_matrix)
        print('transition_matrix\n',self.transition_matrix)
        print('emission_matrix\n',self.emssion_matrix)
        
    def likelihood(self,state,ob_t):
        
        #Modification: convert observation[t] from notes to 2_classes
        chord=self.states[state]  #numeric to chord name
        ob_t=note_2_class(chord,ob_t) #2 class
        
        prob=1
        for x in ob_t:
            prob*=(self.emssion_matrix[state][x])
        return prob
 
    def forward(self,t,j,ob=None):
        if ob is None:
            ob=self.observered
        if t==0:
            return self.initial_matrix[j]*self.likelihood(j,ob[t])
        else:          
            return sum([self.forward(t-1,i,ob)*self.transition_matrix[i][j] for i in range(len(self.states))]) * self.likelihood(j,ob[t])
          
            
    def backward(self,t,i,ob=None):
        if ob is None:
            ob=self.observered
        if t==len(ob)-1:
            return 1
        else:
            return sum([self.transition_matrix[i][j]*self.likelihood(j,ob[t+1])*self.backward(t+1,j,ob) for j in range(len(self.states))])
           
    def probit_at_i(self,t,i,ob=None):#Gamma γt(i) = P(qt = i|O,λ)      
        if ob is None:
            ob=self.observered
        numerator=self.forward(t,i,ob)*self.backward(t,i,ob)#sum probability of all path passing through state[i] at time t
        denominator=sum([self.forward(t,j,ob)*self.backward(t,j,ob) for j in range(len(self.states))]) #prob of passing through  ALL_state at time t
        return numerator/denominator
    
    def probit_transit_i_j(self,t,i,j,ob=None):#epsilon ξt(i, j) = P(qt = i,qt+1 = j|O,λ)
        if ob is None:
            ob=self.observered
        numerator=self.forward(t,i,ob)*self.transition_matrix[i][j]*self.likelihood(j,ob[t+1])*self.backward(t+1,j,ob)#sum probability of all path transit from state[i] to state[j] at time t
        denominator=sum([sum([self.forward(t,m,ob)*self.transition_matrix[m][n]*self.likelihood(n,ob[t+1])*self.backward(t+1,n,ob) for n in range(len(self.states))]) for m in range(len(self.states))]) #prob of ALL transition combination at time t
        return (numerator/denominator)
            
    def train(self,obs=None,epochs=2):
        #O:observed values
        #λ:model parameters
         
        if obs is None:
            obs=self.observered
        
        for epoch in range(epochs):
            for ob in obs:
                
                #initial matrix
                for i in range(len(self.states)):
                    self.initial_matrix[i]=self.probit_at_i(0,i,ob)
                #transition matrix
                for i, j in itertools.product(range(len(self.states)),range(len(self.states))):
                    self.transition_matrix[i][j]=sum([self.probit_transit_i_j(t,i,j,ob) for t in range(len(ob)-1)])/sum([self.probit_at_i(t,i,ob) for t in range(len(ob)-1)])
                #emission matrix
                for j, k in itertools.product(range(len(self.states)),range(len(self.values))):   
                    total=0
                    
                    #Modification: convert notes to 2 classes (outside or inside given chord)
                    chord=self.states[j]  #numeric to chord name
                    ob_2_class=[note_2_class(chord,ob_t) for ob_t in ob] #2 class

                    for t in range(len(ob)):
                        if k in ob_2_class[t]:
                            #Modification: multiple by how many times do k appear at timestamp t
                            total+=self.probit_at_i(t,j,ob)*ob_2_class[t].count(k)  
                            
                    #Modification: multiple by len(ob[t]), which is the total length of notes at timestamp t                    
                    self.emssion_matrix[j][k]=total/sum([self.probit_at_i(t,j,ob)*len(ob[t]) for t in range(len(ob))])  
                    
                    #smoothing
                    if self.emssion_matrix[j][k]==0:
                        self.emssion_matrix[j][k]=0.0000000001

In [4]:
test=HMM(2,2,["CMajorI", "DMajorI"],["outside chord","inside chord"])

In [5]:
test.train([[[10,2,4], [5,0,2], [3,1], [0], [1], [0,1,0]],[[2], [0,3], [3,1], [10], [2], [1]],[[2,0], [10,4], [12,5], [12], [11], [0,1]]],10)

In [6]:
test.debug()

initial_matrix
 [1. 0.]
transition_matrix
 [[0.14002659 0.9442555 ]
 [0.35881461 0.65722131]]
emission_matrix
 [[6.97467846e-01 3.34673677e-01]
 [1.00000000e+00 1.00000000e-10]]
